# Cats Vs Dogs
I'm going to use [Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/) to finetune a pre-existing model to differentiate between Cats and Dogs.

## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models, transforms, datasets
import time
%matplotlib inline

In [2]:
torch.__version__

'2.0.1'

In [3]:
import sys
sys.version

'3.11.4 (v3.11.4:d2340ef257, Jun  6 2023, 19:15:51) [Clang 13.0.0 (clang-1300.0.29.30)]'

**This file is intended to run on Google Colab**

Check if GPU is available, if not change the runtime

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using gpu: %s ' % torch.cuda.is_available())

Using gpu: False 


## Download the Data

The data from [Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/) consists of two files: images.tar.gz and annotations.tar.gz

In [5]:
%pwd

'/Users/jmello/Projects/CatDog'

In [6]:
%mkdir data
%cd data

/Users/jmello/Projects/CatDog/data


In [1]:
!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

--2023-06-27 14:20:01--  http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz [following]
--2023-06-27 14:20:01--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz [following]
--2023-06-27 14:20:02--  https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 301 Moved Perman

Extract the downloaded files

In [2]:
!tar zxvf images.tar.gz
!tar zxvf annotations.tar.gz

x images/
x images/boxer_16.jpg
x images/chihuahua_165.jpg
x images/pug_183.jpg
x images/english_setter_1.jpg
x images/chihuahua_170.jpg
x images/english_cocker_spaniel_17.jpg
x images/samoyed_39.jpg
x images/Egyptian_Mau_62.jpg
x images/samoyed_36.jpg
x images/german_shorthaired_3.jpg
x images/Ragdoll_183.jpg
x images/British_Shorthair_64.jpg
x images/american_pit_bull_terrier_57.jpg
x images/beagle_120.jpg
x images/american_bulldog_174.jpg
x images/chihuahua_101.jpg
x images/shiba_inu_136.jpg
x images/Abyssinian_136.jpg
x images/Siamese_201.jpg
x images/Abyssinian_85.jpg
x images/saint_bernard_145.jpg
x images/Siamese_63.jpg
x images/leonberger_164.jpg
x images/Maine_Coon_126.jpg
x images/samoyed_51.jpg
x images/Birman_15.jpg
x images/english_cocker_spaniel_181.jpg
x images/english_cocker_spaniel_128.jpg
x images/leonberger_133.jpg
x images/english_cocker_spaniel_6.jpg
x images/miniature_pinscher_119.jpg
x images/american_pit_bull_terrier_27.jpg
x images/Abyssinian_37.jpg
x images/Bo

x images/basset_hound_37.jpg
x images/pomeranian_71.jpg
x images/Bombay_208.jpg
x images/samoyed_187.jpg
x images/staffordshire_bull_terrier_123.jpg
x images/Ragdoll_7.jpg
x images/english_setter_26.jpg
x images/english_cocker_spaniel_177.jpg
x images/american_bulldog_217.jpg
x images/Sphynx_53.jpg
x images/pug_88.jpg
x images/shiba_inu_212.jpg
x images/havanese_71.jpg
x images/chihuahua_38.jpg
x images/basset_hound_67.jpg
x images/wheaten_terrier_18.jpg
x images/keeshond_8.jpg
x images/great_pyrenees_48.jpg
x images/chihuahua_180.jpg
x images/shiba_inu_161.jpg
x images/keeshond_88.jpg
x images/beagle_186.jpg
x images/german_shorthaired_7.jpg
x images/Ragdoll_170.jpg
x images/samoyed_168.jpg
x images/miniature_pinscher_106.jpg
x images/beagle_97.jpg
x images/boxer_156.jpg
x images/leonberger_67.jpg
x images/Egyptian_Mau_224.jpg
x images/Birman_151.jpg
x images/boxer_64.jpg
x images/american_bulldog_25.jpg
x images/english_cocker_spaniel_165.jpg
x images/Abyssinian_9.jpg
x images/scotti

x images/Ragdoll_184.jpg
x images/great_pyrenees_106.jpg
x images/Ragdoll_92.jpg
x images/leonberger_144.jpg
x images/Bengal_148.jpg
x images/scottish_terrier_13.jpg
x images/german_shorthaired_124.jpg
x images/British_Shorthair_276.jpg
x images/Maine_Coon_269.jpg
x images/Ragdoll_112.jpg
x images/samoyed_40.jpg
x images/samoyed_77.jpg
x images/japanese_chin_22.jpg
x images/havanese_152.jpg
x images/samoyed_105.jpg
x images/Siamese_182.jpg
x images/keeshond_177.jpg
x images/great_pyrenees_145.jpg
x images/yorkshire_terrier_176.jpg
x images/basset_hound_14.jpg
x images/pug_63.jpg
x images/Persian_156.jpg
x images/Bengal_75.jpg
x images/Ragdoll_164.jpg
x images/scottish_terrier_98.jpg
x images/beagle_22.jpg
x images/english_cocker_spaniel_142.jpg
x images/Maine_Coon_242.jpg
x images/keeshond_144.jpg
x images/great_pyrenees_179.jpg
x images/scottish_terrier_123.jpg
x images/British_Shorthair_86.jpg
x images/Abyssinian_51.jpg
x images/shiba_inu_16.jpg
x images/samoyed_2.jpg
x images/german

x images/saint_bernard_94.jpg
x images/staffordshire_bull_terrier_6.jpg
x images/Persian_160.jpg
x images/pomeranian_192.jpg
x images/leonberger_182.jpg
x images/Abyssinian_45.jpg
x images/great_pyrenees_107.jpg
x images/english_cocker_spaniel_67.jpg
x images/shiba_inu_182.jpg
x images/Egyptian_Mau_167.jpg
x images/english_setter_127.jpg
x images/miniature_pinscher_43.jpg
x images/american_pit_bull_terrier_151.jpg
x images/american_bulldog_23.jpg
x images/Siamese_40.jpg
x images/shiba_inu_71.jpg
x images/english_setter_145.jpg
x images/Egyptian_Mau_109.jpg
x images/yorkshire_terrier_183.jpg
x images/Abyssinian_14.jpg
x images/Bombay_55.jpg
x images/shiba_inu_81.jpg
x images/staffordshire_bull_terrier_21.jpg
x images/wheaten_terrier_190.jpg
x images/chihuahua_99.jpg
x images/great_pyrenees_126.jpg
x images/Ragdoll_56.jpg
x images/yorkshire_terrier_27.jpg
x images/pug_199.jpg
x images/great_pyrenees_185.jpg
x images/shiba_inu_120.jpg
x images/pug_140.jpg
x images/newfoundland_72.jpg
x im

x images/newfoundland_80.jpg
x images/chihuahua_175.jpg
x images/japanese_chin_119.jpg
x images/Persian_169.jpg
x images/Persian_41.jpg
x images/samoyed_72.jpg
x images/Russian_Blue_220.jpg
x images/Maine_Coon_201.jpg
x images/japanese_chin_11.jpg
x images/german_shorthaired_8.jpg
x images/havanese_162.jpg
x images/english_cocker_spaniel_141.jpg
x images/Bombay_204.jpg
x images/Bombay_137.jpg
x images/boxer_48.jpg
x images/British_Shorthair_40.jpg
x images/miniature_pinscher_40.jpg
x images/scottish_terrier_57.jpg
x images/american_pit_bull_terrier_42.jpg
x images/Russian_Blue_162.jpg
x images/newfoundland_132.jpg
x images/leonberger_11.jpg
x images/basset_hound_181.jpg
x images/english_setter_131.jpg
x images/yorkshire_terrier_73.jpg
x images/newfoundland_33.jpg
x images/staffordshire_bull_terrier_33.jpg
x images/samoyed_199.jpg
x images/Bengal_199.jpg
x images/Bengal_160.jpg
x images/Siamese_59.jpg
x images/Birman_181.jpg
x images/scottish_terrier_173.jpg
x images/staffordshire_bull_

...

x annotations/trimaps/shiba_inu_194.png
x annotations/trimaps/shiba_inu_195.png
x annotations/trimaps/shiba_inu_196.png
x annotations/trimaps/shiba_inu_198.png
x annotations/trimaps/shiba_inu_199.png
x annotations/trimaps/shiba_inu_20.png
x annotations/trimaps/shiba_inu_200.png
x annotations/trimaps/shiba_inu_201.png
x annotations/trimaps/shiba_inu_202.png
x annotations/trimaps/shiba_inu_203.png
x annotations/trimaps/shiba_inu_204.png
x annotations/trimaps/shiba_inu_205.png
x annotations/trimaps/shiba_inu_206.png
x annotations/trimaps/shiba_inu_207.png
x annotations/trimaps/shiba_inu_208.png
x annotations/trimaps/shiba_inu_209.png
x annotations/trimaps/shiba_inu_21.png
x annotations/trimaps/shiba_inu_210.png
x annotations/trimaps/shiba_inu_212.png
x annotations/trimaps/shiba_inu_22.png
x annotations/trimaps/shiba_inu_23.png
x annotations/trimaps/shiba_inu_24.png
x annotations/trimaps/shiba_inu_25.png
x annotations/trimaps/shiba_inu_26.png
x annotations/trimaps/shiba_inu_27.png
x annota

x annotations/trimaps/Sphynx_229.png
x annotations/trimaps/Sphynx_23.png
x annotations/trimaps/Sphynx_230.png
x annotations/trimaps/Sphynx_231.png
x annotations/trimaps/Sphynx_232.png
x annotations/trimaps/Sphynx_233.png
x annotations/trimaps/Sphynx_234.png
x annotations/trimaps/Sphynx_235.png
x annotations/trimaps/Sphynx_236.png
x annotations/trimaps/Sphynx_237.png
x annotations/trimaps/Sphynx_238.png
x annotations/trimaps/Sphynx_239.png
x annotations/trimaps/Sphynx_24.png
x annotations/trimaps/Sphynx_240.png
x annotations/trimaps/Sphynx_241.png
x annotations/trimaps/Sphynx_242.png
x annotations/trimaps/Sphynx_243.png
x annotations/trimaps/Sphynx_244.png
x annotations/trimaps/Sphynx_245.png
x annotations/trimaps/Sphynx_246.png
x annotations/trimaps/Sphynx_247.png
x annotations/trimaps/Sphynx_248.png
x annotations/trimaps/Sphynx_249.png
x annotations/trimaps/Sphynx_25.png
x annotations/trimaps/Sphynx_250.png
x annotations/trimaps/Sphynx_251.png
x annotations/trimaps/Sphynx_252.png
x an

x annotations/trimaps/staffordshire_bull_terrier_33.png
x annotations/trimaps/staffordshire_bull_terrier_34.png
x annotations/trimaps/staffordshire_bull_terrier_35.png
x annotations/trimaps/staffordshire_bull_terrier_36.png
x annotations/trimaps/staffordshire_bull_terrier_37.png
x annotations/trimaps/staffordshire_bull_terrier_38.png
x annotations/trimaps/staffordshire_bull_terrier_39.png
x annotations/trimaps/staffordshire_bull_terrier_4.png
x annotations/trimaps/staffordshire_bull_terrier_40.png
x annotations/trimaps/staffordshire_bull_terrier_41.png
x annotations/trimaps/staffordshire_bull_terrier_42.png
x annotations/trimaps/staffordshire_bull_terrier_43.png
x annotations/trimaps/staffordshire_bull_terrier_44.png
x annotations/trimaps/staffordshire_bull_terrier_45.png
x annotations/trimaps/staffordshire_bull_terrier_46.png
x annotations/trimaps/staffordshire_bull_terrier_47.png
x annotations/trimaps/staffordshire_bull_terrier_48.png
x annotations/trimaps/staffordshire_bull_terrier_

x annotations/trimaps/wheaten_terrier_44.png
x annotations/trimaps/wheaten_terrier_45.png
x annotations/trimaps/wheaten_terrier_46.png
x annotations/trimaps/wheaten_terrier_47.png
x annotations/trimaps/wheaten_terrier_48.png
x annotations/trimaps/wheaten_terrier_49.png
x annotations/trimaps/wheaten_terrier_5.png
x annotations/trimaps/wheaten_terrier_50.png
x annotations/trimaps/wheaten_terrier_51.png
x annotations/trimaps/wheaten_terrier_52.png
x annotations/trimaps/wheaten_terrier_53.png
x annotations/trimaps/wheaten_terrier_54.png
x annotations/trimaps/wheaten_terrier_55.png
x annotations/trimaps/wheaten_terrier_56.png
x annotations/trimaps/wheaten_terrier_57.png
x annotations/trimaps/wheaten_terrier_58.png
x annotations/trimaps/wheaten_terrier_59.png
x annotations/trimaps/wheaten_terrier_6.png
x annotations/trimaps/wheaten_terrier_60.png
x annotations/trimaps/wheaten_terrier_61.png
x annotations/trimaps/wheaten_terrier_62.png
x annotations/trimaps/wheaten_terrier_63.png
x annotation

x annotations/trimaps/yorkshire_terrier_50.png
x annotations/trimaps/yorkshire_terrier_51.png
x annotations/trimaps/yorkshire_terrier_52.png
x annotations/trimaps/yorkshire_terrier_53.png
x annotations/trimaps/yorkshire_terrier_54.png
x annotations/trimaps/yorkshire_terrier_55.png
x annotations/trimaps/yorkshire_terrier_56.png
x annotations/trimaps/yorkshire_terrier_57.png
x annotations/trimaps/yorkshire_terrier_58.png
x annotations/trimaps/yorkshire_terrier_59.png
x annotations/trimaps/yorkshire_terrier_6.png
x annotations/trimaps/yorkshire_terrier_60.png
x annotations/trimaps/yorkshire_terrier_61.png
x annotations/trimaps/yorkshire_terrier_62.png
x annotations/trimaps/yorkshire_terrier_63.png
x annotations/trimaps/yorkshire_terrier_64.png
x annotations/trimaps/yorkshire_terrier_65.png
x annotations/trimaps/yorkshire_terrier_66.png
x annotations/trimaps/yorkshire_terrier_67.png
x annotations/trimaps/yorkshire_terrier_68.png
x annotations/trimaps/yorkshire_terrier_69.png
x annotations/

# Data Wrangling

In [3]:
%ls

CatVsDog.ipynb      annotations.tar.gz  images/
annotations/        data/               images.tar.gz


In [6]:
%ls annotations

README        test.txt      trimaps/
list.txt      trainval.txt  xmls/


In [4]:
!head annotations/test.txt

Abyssinian_201 1 1 1
Abyssinian_202 1 1 1
Abyssinian_204 1 1 1
Abyssinian_205 1 1 1
Abyssinian_206 1 1 1
Abyssinian_207 1 1 1
Abyssinian_20 1 1 1
Abyssinian_210 1 1 1
Abyssinian_212 1 1 1
Abyssinian_213 1 1 1


In [5]:
!head annotations/trainval.txt

Abyssinian_100 1 1 1
Abyssinian_101 1 1 1
Abyssinian_102 1 1 1
Abyssinian_103 1 1 1
Abyssinian_104 1 1 1
Abyssinian_105 1 1 1
Abyssinian_106 1 1 1
Abyssinian_107 1 1 1
Abyssinian_108 1 1 1
Abyssinian_109 1 1 1


In [7]:
%mkdir test trainval

In [8]:
%ls

CatVsDog.ipynb      annotations.tar.gz  images/             test/
annotations/        data/               images.tar.gz       trainval/


In [1]:
def check_dir(dir_path):
    dir_path = dir_path.replace('//','/')
    os.makedirs(dir_path, exist_ok=True)

In [2]:
import re
import os
import shutil
#Load Regex
pat = re.compile(r'_\d')
pwd = os.getcwd()
#define data wrangler function to format file system in desired order
def data_wrangle(folder, lowercase):
    #iterate through test file for list images to save to test dir
    with open(f'./annotations/{folder}.txt') as fp:
        line = fp.readline()
        while line:
            f,_,_,_ = line.split(' ')
            res,_ = pat.split(f)
            line = fp.readline()
            path = os.path.join(pwd,f"{folder}/",res)
            src = os.path.join(pwd,"images/",f"{f}.jpg")
            if not lowercase and not res.islower():
                check_dir(path)
                print("path '%s' created" %path)
                path = os.path.join(pwd,f"{folder}/{res}",f"{f}.jpg")
                shutil.copy(src, path)
                print(f"file from {src} copied to {path}")
            elif lowercase and res.islower():
                check_dir(path)
                print("path '%s' created" %path)
                path = os.path.join(pwd,f"{folder}/{res}",f"{f}.jpg")
                shutil.copy(src, path)
                print(f"file from {src} copied to {path}")

/Users/jmello/Projects/CatDog/CatVsDog


In [3]:
data_wrangle("test",False)

path '/Users/jmello/Projects/CatDog/CatVsDog/test/Abyssinian' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Abyssinian_201.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/Abyssinian/Abyssinian_201.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/Abyssinian' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Abyssinian_202.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/Abyssinian/Abyssinian_202.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/Abyssinian' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Abyssinian_204.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/Abyssinian/Abyssinian_204.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/Abyssinian' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Abyssinian_205.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/Abyssinian/Abyssinian_205.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/Abyssinian' created
file from /Use

file from /Users/jmello/Projects/CatDog/CatVsDog/images/Bombay_72.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/Bombay/Bombay_72.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/Bombay' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Bombay_73.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/Bombay/Bombay_73.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/Bombay' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Bombay_74.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/Bombay/Bombay_74.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/Bombay' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Bombay_75.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/Bombay/Bombay_75.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/Bombay' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Bombay_76.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/Bombay/Bombay_76.jpg
path '/User

file from /Users/jmello/Projects/CatDog/CatVsDog/images/Ragdoll_33.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/Ragdoll/Ragdoll_33.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/Ragdoll' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Ragdoll_34.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/Ragdoll/Ragdoll_34.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/Ragdoll' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Ragdoll_35.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/Ragdoll/Ragdoll_35.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/Ragdoll' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Ragdoll_36.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/Ragdoll/Ragdoll_36.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/Ragdoll' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Ragdoll_37.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/Ragdoll/Ragdoll

In [4]:
data_wrangle("test",True)

path '/Users/jmello/Projects/CatDog/CatVsDog/test/american_bulldog' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/american_bulldog_203.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/american_bulldog/american_bulldog_203.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/american_bulldog' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/american_bulldog_205.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/american_bulldog/american_bulldog_205.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/american_bulldog' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/american_bulldog_207.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/american_bulldog/american_bulldog_207.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/american_bulldog' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/american_bulldog_208.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/american_bulldog/american_bull

file from /Users/jmello/Projects/CatDog/CatVsDog/images/beagle_75.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/beagle/beagle_75.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/beagle' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/beagle_76.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/beagle/beagle_76.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/beagle' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/beagle_77.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/beagle/beagle_77.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/beagle' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/beagle_78.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/beagle/beagle_78.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/beagle' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/beagle_79.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/beagle/beagle_79.jpg
path '/User

file from /Users/jmello/Projects/CatDog/CatVsDog/images/german_shorthaired_193.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/german_shorthaired/german_shorthaired_193.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/german_shorthaired' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/german_shorthaired_194.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/german_shorthaired/german_shorthaired_194.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/german_shorthaired' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/german_shorthaired_195.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/german_shorthaired/german_shorthaired_195.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/german_shorthaired' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/german_shorthaired_196.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/german_shorthaired/german_shorthaired_196.jpg
path '/Users/jmello/Projects/CatDo

file from /Users/jmello/Projects/CatDog/CatVsDog/images/keeshond_66.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/keeshond/keeshond_66.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/keeshond' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/keeshond_67.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/keeshond/keeshond_67.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/keeshond' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/keeshond_68.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/keeshond/keeshond_68.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/keeshond' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/keeshond_69.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/keeshond/keeshond_69.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/keeshond' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/keeshond_6.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test

file from /Users/jmello/Projects/CatDog/CatVsDog/images/pomeranian_96.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/pomeranian/pomeranian_96.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/pomeranian' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/pomeranian_97.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/pomeranian/pomeranian_97.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/pomeranian' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/pomeranian_98.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/pomeranian/pomeranian_98.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/pomeranian' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/pomeranian_99.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/pomeranian/pomeranian_99.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/pomeranian' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/pomeranian_9.jpg copied to /Users/jm

file from /Users/jmello/Projects/CatDog/CatVsDog/images/shiba_inu_59.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/shiba_inu/shiba_inu_59.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/shiba_inu' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/shiba_inu_5.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/shiba_inu/shiba_inu_5.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/shiba_inu' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/shiba_inu_60.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/shiba_inu/shiba_inu_60.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/shiba_inu' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/shiba_inu_61.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/test/shiba_inu/shiba_inu_61.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/test/shiba_inu' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/shiba_inu_62.jpg copied to /Users/jmello/Projects/CatD

In [5]:
data_wrangle("trainval",False)

path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Abyssinian' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Abyssinian_100.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Abyssinian/Abyssinian_100.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Abyssinian' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Abyssinian_101.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Abyssinian/Abyssinian_101.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Abyssinian' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Abyssinian_102.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Abyssinian/Abyssinian_102.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Abyssinian' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Abyssinian_103.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Abyssinian/Abyssinian_103.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainva

file from /Users/jmello/Projects/CatDog/CatVsDog/images/Bengal_138.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Bengal/Bengal_138.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Bengal' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Bengal_139.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Bengal/Bengal_139.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Bengal' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Bengal_13.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Bengal/Bengal_13.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Bengal' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Bengal_140.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Bengal/Bengal_140.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Bengal' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Bengal_141.jpg copied to /Users/jmello/Projects/CatDog/CatVsDo

file from /Users/jmello/Projects/CatDog/CatVsDog/images/Russian_Blue_135.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Russian_Blue/Russian_Blue_135.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Russian_Blue' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Russian_Blue_136.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Russian_Blue/Russian_Blue_136.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Russian_Blue' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Russian_Blue_137.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Russian_Blue/Russian_Blue_137.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Russian_Blue' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Russian_Blue_139.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Russian_Blue/Russian_Blue_139.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Russian_Blue' created
file from /Users/jme

file from /Users/jmello/Projects/CatDog/CatVsDog/images/Sphynx_142.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Sphynx/Sphynx_142.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Sphynx' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Sphynx_143.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Sphynx/Sphynx_143.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Sphynx' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Sphynx_144.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Sphynx/Sphynx_144.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Sphynx' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Sphynx_145.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Sphynx/Sphynx_145.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Sphynx' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Sphynx_146.jpg copied to /Users/jmello/Projects/CatDog/CatVs

file from /Users/jmello/Projects/CatDog/CatVsDog/images/Persian_195.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Persian/Persian_195.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Persian' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Persian_196.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Persian/Persian_196.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Persian' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Persian_197.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Persian/Persian_197.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Persian' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Persian_19.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Persian/Persian_19.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Persian' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Persian_1.jpg copied to /Users/jmello/Projects

file from /Users/jmello/Projects/CatDog/CatVsDog/images/Russian_Blue_193.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Russian_Blue/Russian_Blue_193.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Russian_Blue' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Russian_Blue_194.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Russian_Blue/Russian_Blue_194.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Russian_Blue' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Russian_Blue_195.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Russian_Blue/Russian_Blue_195.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Russian_Blue' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/Russian_Blue_196.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/Russian_Blue/Russian_Blue_196.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/Russian_Blue' created
file from /Users/jme

In [6]:
data_wrangle("trainval",True)

path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/american_bulldog' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/american_bulldog_100.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/american_bulldog/american_bulldog_100.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/american_bulldog' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/american_bulldog_101.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/american_bulldog/american_bulldog_101.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/american_bulldog' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/american_bulldog_102.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/american_bulldog/american_bulldog_102.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/american_bulldog' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/american_bulldog_103.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainva

file from /Users/jmello/Projects/CatDog/CatVsDog/images/german_shorthaired_115.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/german_shorthaired/german_shorthaired_115.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/german_shorthaired' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/german_shorthaired_116.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/german_shorthaired/german_shorthaired_116.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/german_shorthaired' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/german_shorthaired_117.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/german_shorthaired/german_shorthaired_117.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/german_shorthaired' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/german_shorthaired_118.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/german_shorthaired/german_shorthaired_118.jpg
path '

file from /Users/jmello/Projects/CatDog/CatVsDog/images/pug_118.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/pug/pug_118.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/pug' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/pug_119.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/pug/pug_119.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/pug' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/pug_11.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/pug/pug_11.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/pug' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/pug_120.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/pug/pug_120.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/pug' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/pug_121.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/pug/pug_121.jpg
path '/Users/jmello/Proj

file from /Users/jmello/Projects/CatDog/CatVsDog/images/american_pit_bull_terrier_165.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/american_pit_bull_terrier/american_pit_bull_terrier_165.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/american_pit_bull_terrier' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/american_pit_bull_terrier_166.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/american_pit_bull_terrier/american_pit_bull_terrier_166.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/american_pit_bull_terrier' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/american_pit_bull_terrier_167.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/american_pit_bull_terrier/american_pit_bull_terrier_167.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/american_pit_bull_terrier' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/american_pit_bull_terrier_168.jpg copied to /Users/jm

file from /Users/jmello/Projects/CatDog/CatVsDog/images/japanese_chin_149.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/japanese_chin/japanese_chin_149.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/japanese_chin' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/japanese_chin_14.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/japanese_chin/japanese_chin_14.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/japanese_chin' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/japanese_chin_150.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/japanese_chin/japanese_chin_150.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/japanese_chin' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/japanese_chin_151.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/japanese_chin/japanese_chin_151.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/japanese_chin' created
file f

file from /Users/jmello/Projects/CatDog/CatVsDog/images/scottish_terrier_184.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/scottish_terrier/scottish_terrier_184.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/scottish_terrier' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/scottish_terrier_185.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/scottish_terrier/scottish_terrier_185.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/scottish_terrier' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/scottish_terrier_186.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/scottish_terrier/scottish_terrier_186.jpg
path '/Users/jmello/Projects/CatDog/CatVsDog/trainval/scottish_terrier' created
file from /Users/jmello/Projects/CatDog/CatVsDog/images/scottish_terrier_187.jpg copied to /Users/jmello/Projects/CatDog/CatVsDog/trainval/scottish_terrier/scottish_terrier_187.jpg
path '/Users/jmello/Projects/CatDog/

# DO NOT RUN ABOVE CODE AGAIN

In [8]:
%ls test

Abyssinian/                 english_setter/
Bengal/                     german_shorthaired/
Birman/                     great_pyrenees/
Bombay/                     havanese/
British_Shorthair/          japanese_chin/
Egyptian_Mau/               keeshond/
Maine_Coon/                 leonberger/
Persian/                    miniature_pinscher/
Ragdoll/                    newfoundland/
Russian_Blue/               pomeranian/
Siamese/                    pug/
Sphynx/                     saint_bernard/
american_bulldog/           samoyed/
american_pit_bull_terrier/  scottish_terrier/
basset_hound/               shiba_inu/
beagle/                     staffordshire_bull_terrier/
boxer/                      wheaten_terrier/
chihuahua/                  yorkshire_terrier/
english_cocker_spaniel/


In [9]:
%ls trainval

Abyssinian/                 english_setter/
Bengal/                     german_shorthaired/
Birman/                     great_pyrenees/
Bombay/                     havanese/
British_Shorthair/          japanese_chin/
Egyptian_Mau/               keeshond/
Maine_Coon/                 leonberger/
Persian/                    miniature_pinscher/
Ragdoll/                    newfoundland/
Russian_Blue/               pomeranian/
Siamese/                    pug/
Sphynx/                     saint_bernard/
american_bulldog/           samoyed/
american_pit_bull_terrier/  scottish_terrier/
basset_hound/               shiba_inu/
beagle/                     staffordshire_bull_terrier/
boxer/                      wheaten_terrier/
chihuahua/                  yorkshire_terrier/
english_cocker_spaniel/
